**Chapter 9 – Unsupervised Learning**

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/HOGENT-ML/course/machinelearning/blob/main/900-unsupervised_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Setup

The regular requirements as in previous notebooks

In [2]:
import sys
from packaging import version
import sklearn

assert sys.version_info >= (3, 7)
assert version.parse(sklearn.__version__) >= version.parse("1.2.1")

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

# Common imports
import numpy as np
import os

In [3]:
sklearn.__version__

'1.2.2'

> Version 1.2.2 of sklearn is absolutely necessary for this chapter.    
> If your version is not sufficient then upgrade through `pip install --upgrade scikit-learn`. 

# Unsupervised learning

Most of the applications of machine learning today are based on supervised
learning (and as a result, this is where most of the investments go to).     
The vast majority of the available data however is unlabeled: we have the input features X, but we do not have the labels y.    

Let's assume you want to setup a quality assurance system on a manufacturing production line.   
You want to create a system that will take a few pictures of each item on a
manufacturing production line and detect which items are defective. You can fairly
easily create a system that will take pictures automatically, and this might give you
thousands of pictures every day. You can then build a reasonably large dataset in just
a few weeks.   
The problem comes when you want to provide labels “defective” or “normal” to all items. 
If you want to train a regular binary classifier that will predict whether an item is defective or not, you will need to label some pictures to use as training set.  
This will generally require human expert knowledge and time. This requires an expensive effort (time and money).   
So it will usually only be done on a small subset of the available pictures. As a
result, the labeled dataset will be quite small, and the classifier’s performance will be
disappointing. Moreover, every time the company makes any change to its products,
the whole process will need to be started over from scratch. Wouldn’t it be great if
the algorithm could just exploit the unlabeled data without needing humans to label
every picture? 

Take a look at the following figure to refresh our mind about the domain of Machine Learning (source: Big data and machine learning for Businesses, Abdul Rahid):   

<img width=50% src="img/machine_learning_areas.png"> 

# Introduction

**Clustering**
The goal is to group similar instances together into clusters.  
Clustering is a great tool for data analysis, customer segmentation, recommender systems, search
engines, image segmentation, semi-supervised learning, dimensionality reduction,
and more.  

**Anomaly detection (also called outlier detection)**  
The objective is to learn what “normal” data looks like, and then use that to
detect abnormal instances. These instances are called anomalies, or outliers, while
the normal instances are called inliers. Anomaly detection is useful in a wide
variety of applications, such as fraud detection, detecting defective products in
manufacturing, identifying new trends in time series, or removing outliers from
a dataset before training another model, which can significantly improve the
performance of the resulting model.  

**Density estimation**  
This is the task of estimating the probability density function (PDF) of the random
process that generated the dataset. Density estimation is commonly used for
anomaly detection: instances located in very low-density regions are likely to be
anomalies. It is also useful for data analysis and visualization.    

We will cover with two clustering algorithms, k-means and
DBSCAN. At the end we’ll discuss Gaussian mixture models and see how they can be used
for density estimation, clustering, and anomaly detection.

# Clustering

**Introduction – Classification _vs_ Clustering**

Look at the image below.
The image on the left is the result of a supervised classification.  
On the right where we want to perform clustering on the iris dataset.  
There are no labels, since we don't know any labels in unsupervised learning.  
<img width=75% src="img/classification_vs_clustering_plot.png"> 

The next figure shows how a Gaussian mixture model that can actually separate these clusters pretty well using all 4 features: petal length & width, and sepal length & width. This code maps each cluster to a class. Instead of hard coding the mapping, the code picks the most common class for each cluster. 96.6% of the plants is assigned to the right cluster.
<img width=50% src="img/clustering_iris_plot.png"> 

## K-Means

**Fit and predict**

Let's train a K-Means clusterer on a dataset of blobs. It will try to find each blob's center and assign each instance to the closest blob:

In [4]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

# extra code – the exact arguments of make_blobs() are not important
blob_centers = np.array([[ 0.2,  2.3], [-1.5 ,  2.3], [-2.8,  1.8],
                         [-2.8,  2.8], [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers, cluster_std=blob_std,
                  random_state=7)

k = 5
kmeans = KMeans(n_clusters=k, random_state=42,n_init="auto") 
y_pred = kmeans.fit_predict(X)

- Note that you have to specify the number of clusters k that the algorithm must find.
- In this example, it is pretty obvious from looking at the data that k should be set to 5,
but in general it is not that easy. 
- We will discuss this shortly.

Now let's plot them:   
<img width=50% src="img/blobs_plot.png"> 

Each instance was assigned to one of the 5 clusters:

In [5]:
y_pred

array([1, 1, 4, ..., 3, 2, 1])

In [6]:
y_pred is kmeans.labels_

True

And the following 5 _centroids_ (i.e., cluster centers) were estimated:

In [7]:
kmeans.cluster_centers_

array([[-1.46893704,  2.28372774],
       [-2.80214068,  1.55162671],
       [ 0.18304455,  2.55635115],
       [-2.79290307,  2.79641063],
       [ 0.23444426,  1.90682419]])

- Note that the `KMeans` instance preserves the labels of the instances it was trained on. 
- Somewhat confusingly, in this context, the _label_ of an instance is the index of the cluster that instance gets assigned to (they are not targets, they are predictions):

In [8]:
kmeans.labels_

array([1, 1, 4, ..., 3, 2, 1])

In [9]:
kmeans.labels_.shape

(2000,)

Of course, we can predict the labels of new instances:

In [10]:
import numpy as np

X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])
kmeans.predict(X_new)

array([4, 4, 3, 3])

**Decision Boundaries**

Let's plot the model's decision boundaries. This gives us a _Voronoi diagram_:  
<img width=50% src="img/voronoi_plot.png"> 

Not bad! Some of the instances near the edges were probably assigned to the wrong cluster, but overall it looks pretty good.

**Hard Clustering _vs_ Soft Clustering**

Rather than arbitrarily choosing the closest cluster for each instance, which is called _hard clustering_, it might be better to measure the distance of each instance to all 5 centroids. This is what the `transform()` method does:

In [11]:
kmeans.transform(X_new).round(2)

array([[1.5 , 2.84, 0.59, 2.9 , 0.25],
       [4.48, 5.82, 2.87, 5.85, 2.77],
       [1.69, 1.46, 3.21, 0.29, 3.41],
       [1.55, 0.97, 3.18, 0.36, 3.29]])

You can verify that this is indeed the Euclidian distance between each instance and each centroid:

In [12]:
# extra code
np.linalg.norm(np.tile(X_new, (1, k)).reshape(-1, k, 2)
               - kmeans.cluster_centers_, axis=2).round(2)

array([[1.5 , 2.84, 0.59, 2.9 , 0.25],
       [4.48, 5.82, 2.87, 5.85, 2.77],
       [1.69, 1.46, 3.21, 0.29, 3.41],
       [1.55, 0.97, 3.18, 0.36, 3.29]])

### The K-Means Algorithm

Please note that the mathematical explanation of the k-means algorithm was touched upon in the course "Mathematics for Machine Learning" earlier in this curriculum.   
The K-Means algorithm is one of the fastest clustering algorithms, and also one of the simplest:
* First initialize $k$ centroids randomly: e.g., $k$ distinct instances are chosen randomly from the dataset and the centroids are placed at their locations.
* Repeat until convergence (i.e., until the centroids stop moving):
    * Assign each instance to the closest centroid.
    * Update the centroids to be the mean of the instances that are assigned to them.

The `KMeans` class uses an optimized initialization technique by default. To get the original K-Means algorithm (for educational purposes only), you must set `init="random"` and `n_init=1`. More on this later in this chapter.

Let's run the K-Means algorithm for 1, 2 and 3 iterations, to see how the centroids move around:

<img width=75% src="img/kmeans_algorithm_plot.png"> 

**K-Means Variability**

In the original K-Means algorithm, the centroids are just initialized randomly, and the algorithm simply runs a single iteration to gradually improve the centroids, as we saw above.

However, one major problem with this approach is that if you run K-Means multiple times (or with different random seeds), it can converge to very different solutions, as you can see : 
<img width=75% src="img/kmeans_variability_plot.png"> 

In [13]:
good_init = np.array([[-3, 3], [-3, 2], [-3, 1], [-1, 2], [0, 2]])
kmeans = KMeans(n_clusters=5, init=good_init, n_init=1, random_state=42)
kmeans.fit(X)

KMeans(init=array([[-3,  3],
       [-3,  2],
       [-3,  1],
       [-1,  2],
       [ 0,  2]]),
       n_clusters=5, n_init=1, random_state=42)

The updated decision boundaries:   
<img width=60% src="img/clustering_decision_boundaries.png"> 

### Inertia

To select the best model, we will need a way to evaluate a K-Mean model's performance. Unfortunately, clustering is an unsupervised task, so we do not have the targets. But at least we can measure the distance between each instance and its centroid. Inertia is the sum of the squared distances between each training instance and its closest centroid:

In [14]:
kmeans.inertia_

211.59853725816836

The `score()` method returns the negative inertia. Why negative? Well, it is because a predictor's `score()` method must always respect the "_greater is better_" rule.

In [15]:
kmeans.score(X)

-211.59853725816836

### Multiple Initializations

So one approach to solve the variability issue is to simply run the K-Means algorithm multiple times with different random initializations, and select the solution that minimizes the inertia.

When you set the `n_init` hyperparameter, Scikit-Learn runs the original algorithm `n_init` times, and selects the solution that minimizes the inertia. By default, Scikit-Learn sets `n_init=10`. In future versions, default value will be "auto"

In [16]:
# extra code
kmeans_rnd_10_inits = KMeans(n_clusters=5, init="random", n_init=10,
                             random_state=2)
kmeans_rnd_10_inits.fit(X)

KMeans(init='random', n_clusters=5, n_init=10, random_state=2)

### Centroid initialization methods

Instead of initializing the centroids entirely randomly, it is preferable to initialize them using the following algorithm, proposed in a [2006 paper](https://goo.gl/eNUPw6) by David Arthur and Sergei Vassilvitskii:
* Take one centroid $c_1$, chosen uniformly at random from the dataset.
* Take a new center $c_i$, choosing an instance $\mathbf{x}_i$ with probability: $D(\mathbf{x}_i)^2$ / $\sum\limits_{j=1}^{m}{D(\mathbf{x}_j)}^2$ where $D(\mathbf{x}_i)$ is the distance between the instance $\mathbf{x}_i$ and the closest centroid that was already chosen. This probability distribution ensures that instances that are further away from already chosen centroids are much more likely be selected as centroids.
* Repeat the previous step until all $k$ centroids have been chosen.

This algorithm is called K-Means++. The rest of the K-Means++ algorithm is just regular K-Means. With this initialization, the K-Means algorithm is much less likely to converge to a suboptimal solution, so it is possible to reduce `n_init` considerably. Most of the time, this largely compensates for the additional complexity of the initialization process.

To set the initialization to K-Means++, simply set `init="k-means++"` (this is actually the default):

### Accelerated K-Means

The K-Means algorithm can sometimes be accelerated by avoiding many unnecessary distance calculations: this is achieved by exploiting the triangle inequality (given three points A, B and C, the distance AC is always such that AC ≤ AB + BC) and by keeping track of lower and upper bounds for distances between instances and centroids (see this [2003 paper](https://www.aaai.org/Papers/ICML/2003/ICML03-022.pdf) by Charles Elkan for more details).

For Elkan's variant of K-Means, use `algorithm="elkan"`. For regular KMeans, use `algorithm="full"`. The default is `"auto"`, which uses the full algorithm since Scikit-Learn 1.1 (it used Elkan's algorithm before that).

### Finding the optimal number of clusters

What if the number of clusters was set to a lower or greater value than 5? 

In [17]:
kmeans_k3 = KMeans(n_clusters=3, init="random", random_state=2)
kmeans_k8 = KMeans(n_clusters=8, init="random", random_state=2)
kmeans_k3.fit(X)
kmeans_k8.fit(X)

c:\Users\jcor864\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\jcor864\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(init='random', random_state=2)

 <img width=60% src="img/bad_n_clusters_plot.png"> 

Ouch, these two models don't look great. What about their inertias?

In [26]:
kmeans_k3.inertia_

653.2167190021553

In [27]:
kmeans_k8.inertia_

130.01748738325324

- We cannot simply take the value of $k$ that minimizes the inertia, since it keeps getting lower as we increase $k$. Indeed, the more clusters there are, the closer each instance will be to its closest centroid, and therefore the lower the inertia will be.  
- This is another example of overfitting.    
- However, we can plot the inertia as a function of $k$ and analyze the resulting curve:

  
<img width=60% src="img/inertia_vs_k_plot.png"> 


- As you can see, there is an elbow at $k=4$, which means that less clusters than that would be bad, and more clusters would not help much and might cut clusters in half. 
- So $k=4$ is a pretty good choice. 
- Of course in this example it is not perfect since it means that the two blobs in the lower left will be considered as just a single cluster, but it's a pretty good clustering nonetheless.  
  
<img width=60% src="img/clusters_k4.png"> 


- Another approach is to look at the _silhouette score_, which is the mean _silhouette coefficient_ over all the instances. 
- An instance's **silhouette coefficient** is equal to $(b - a) / max(a, b)$ 
  - where $a$ is the mean distance to the other instances in the same cluster (it is the _mean intra-cluster distance_)
  - and $b$ is the _mean nearest-cluster distance_, that is the mean distance to the instances of the next closest cluster (defined as the one that minimizes $b$, excluding the instance's own cluster). 
- The silhouette coefficient can vary between -1 and +1: 
  - a coefficient close to +1 means that the instance is well inside its own cluster and far from other clusters, 
  - a coefficient close to 0 means that it is close to a cluster boundary
  - a coefficient close to -1 means that the instance may have been assigned to the wrong cluster.

Let's plot the silhouette score as a function of $k$:

In [18]:
from sklearn.metrics import silhouette_score

In [20]:
silhouette_score(X, kmeans.labels_) # X = all instances in the dataset, kmeans.labels_ = corresponding cluster labels

0.655517642572828

Let’s compare the silhouette scores for different numbers of clusters:

<img width=60% src="img/silhouette_score_vs_k_plot.png"> 

As you can see, this visualization is much richer than the previous one: in particular, although it confirms that $k=4$ is a very good choice, but it also underlines the fact that $k=5$ is quite good as well.

An even more informative visualization is given when you plot every instance's silhouette coefficient, sorted by the cluster they are assigned to and by the value of the coefficient. This is called a _silhouette diagram_:   
<img width=60% src="img/silhouette_analysis_plot.png"> 

As you can see, $k=5$ looks like the best option here, as all clusters are roughly the same size, and they all cross the dashed line, which represents the mean silhouette score.

## Limits of K-Means

Let's generate a more difficult dataset, with elongated blobs and varying densities, and show that K-Means struggles to cluster it correctly:     
<img width=60% src="img/bad_kmeans_plot.png">  

- As you can see, neither of these solutions is any good. 
- The solution on the left is better, but it still chops off 25% of the middle cluster and assigns it to the cluster on the right. 
- The solution on the right is just terrible, even though its inertia is lower. 
- So, depending on the data, different clustering algorithms may perform better. 
- On these types of elliptical clusters, _Gaussian mixture_ (see further) models work great.

## Using Clustering for Image Segmentation

Take a look af the following ladybug image:    

<img width=40% src="img/ladybug.png">  

In [22]:
import PIL
filepath = "img/ladybug.png"
image = np.asarray(PIL.Image.open(filepath))
image.shape

(533, 800, 3)

533 pixels  high, 800 pixels wide, 3 color channels.

In [38]:
X = image.reshape(-1, 3)  # reshape to a 2D array with 3 columns (R,G,B)
print("X.shape = ")
print(X.shape)
kmeans = KMeans(n_clusters=8, random_state=42, n_init=10).fit(X)
print("8 cluster centers = ")
print(kmeans.cluster_centers_)
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
print("first 10 pixels of segmented image before reshape (img replace by center)= ")
print(segmented_img[:10,:])
segmented_img = segmented_img.reshape(image.shape)
print("first 10 pixels of segmented image after reshape to original shape = ")
print(segmented_img[:10,:])

X.shape = 
(426400, 3)
8 cluster centers = 
[[250.80339083 238.65268971   6.56603259]
 [  5.83448628  28.22616668   1.47552886]
 [ 55.88206518  98.62346563  14.79208777]
 [193.20797818  54.13433345  11.37470167]
 [ 25.47759236  64.83281062   4.31953567]
 [156.22887957 160.67579389  98.81396046]
 [ 94.89017515 133.51803159  40.11328915]
 [225.57151644 185.02493849   8.77650414]]
first 10 pixels of segmented imgage before reshape (img replace by center)= 
[[ 5.83448628 28.22616668  1.47552886]
 [ 5.83448628 28.22616668  1.47552886]
 [ 5.83448628 28.22616668  1.47552886]
 [ 5.83448628 28.22616668  1.47552886]
 [ 5.83448628 28.22616668  1.47552886]
 [ 5.83448628 28.22616668  1.47552886]
 [ 5.83448628 28.22616668  1.47552886]
 [ 5.83448628 28.22616668  1.47552886]
 [ 5.83448628 28.22616668  1.47552886]
 [ 5.83448628 28.22616668  1.47552886]]
first 10 pixels of segmented imgage after reshape to original shape = 
[[[  5.83448628  28.22616668   1.47552886]
  [  5.83448628  28.22616668   1.4755

The images below compare the original image with it's replacement by the center in case of 10, 8, 6, 4, 2 centers. 

How many different colors at most do we have have in the original image? 

<img width=75% src="img/image_segmentation_plot.png">  

## Using Clustering for Semi-Supervised Learning

Another use case for clustering is semi-supervised learning, when we have plenty of unlabeled instances and very few labeled instances.

Let's tackle the _digits dataset_ which is a simple MNIST-like dataset containing 1,797 grayscale 8×8 images representing digits 0 to 9.

In [40]:
from sklearn.datasets import load_digits

X_digits, y_digits = load_digits(return_X_y=True)
X_train, y_train = X_digits[:1400], y_digits[:1400]
X_test, y_test = X_digits[1400:], y_digits[1400:]

Let's look at the performance of a logistic regression model when we only have 50 labeled instances:

In [41]:
from sklearn.linear_model import LogisticRegression

n_labeled = 50
log_reg = LogisticRegression(max_iter=10_000)
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

LogisticRegression(max_iter=10000)

In [42]:
log_reg.score(X_test, y_test)

0.7481108312342569

- The model’s accuracy is just 74.8%. 
- That’s not great: indeed, if you try training the model on the full training set, you will find that it will reach about 90.7% accuracy:

In [44]:
# extra code – measure the accuracy when we use the whole training set
log_reg_full = LogisticRegression(max_iter=10_000)
log_reg_full.fit(X_train, y_train)
log_reg_full.score(X_test, y_test)

0.906801007556675

- Let's see how we can do better. 
- First, let's cluster the training set into 50 clusters.
- Then for each cluster let's find the image closest to the centroid. 
- We will call these images the representative images:

In [45]:
k = 50
kmeans = KMeans(n_clusters=k, random_state=42,n_init=10)
X_digits_dist = kmeans.fit_transform(X_train) # array with distances to each cluster center for whole training set
representative_digit_idx = X_digits_dist.argmin(axis=0)  # cluster label for each training instance
X_representative_digits = X_train[representative_digit_idx] # array with representative digits for each cluster

Now let's plot these representative images and label them manually:

<img width=60% src="img/representative_images_plot.png">  

In [46]:
y_representative_digits = np.array([
    1, 3, 6, 0, 7, 9, 2, 4, 8, 9,
    5, 4, 7, 1, 2, 6, 1, 2, 5, 1,
    4, 1, 3, 3, 8, 8, 2, 5, 6, 9,
    1, 4, 0, 6, 8, 3, 4, 6, 7, 2,
    4, 1, 0, 7, 5, 1, 9, 9, 3, 7
])  # we manually set the labels for each representative digit

Now we have a dataset with just 50 labeled instances, but instead of being completely random instances, each of them is a representative image of its cluster. Let's see if the performance is any better:

In [47]:
log_reg = LogisticRegression(max_iter=10_000)
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(X_test, y_test)

0.8488664987405542

Wow! We jumped from 74.8% accuracy to 84.9%, although we are still only training the model on 50 instances. Since it's often costly and painful to label instances, especially when it has to be done manually by experts, it's a good idea to make them label representative instances rather than just random instances.

But perhaps we can go one step further: what if we propagated the labels to all the other instances in the same cluster?

In [55]:
y_train_propagated = np.empty(len(X_train), dtype=np.int64)
for i in range(k):
    y_train_propagated[kmeans.labels_ == i] = y_representative_digits[i]

In [56]:
log_reg = LogisticRegression(max_iter=10_000)
log_reg.fit(X_train, y_train_propagated)

LogisticRegression(max_iter=10000)

In [57]:
log_reg.score(X_test, y_test)

0.8942065491183879

- We got another significant accuracy boost! 
- Note we labeled manually only 50 instances from a total of 1400, the rest was labeled automatically. 
- Let's see if we can do even better by ignoring the 1% instances that are farthest from their cluster center: this should eliminate some outliers:

In [58]:
# extra code 
percentile_closest = 99 #try to play around with this parameter to see what happens

X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1

partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

In [60]:
log_reg = LogisticRegression(max_iter=10_000)
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
log_reg.score(X_test, y_test)

0.9093198992443325

Wow, another accuracy boost! We have even slightly surpassed the performance we got by training on the fully labeled training set!

Our propagated labels are actually pretty good: their accuracy is about 97.6%:

In [61]:
(y_train_partially_propagated == y_train[partially_propagated]).mean()

0.9755555555555555

#### Active Learning

To continue improving your model and your training set, the next step could be to do a few rounds of active learning, which is when **a human expert interacts with the learning algorithm, providing labels for specific instances when the algorithm requests them**. 
There are many different strategies for active learning, but one of the most common ones is called uncertainty sampling. 
Here is how it works:
1. The model is trained on the labeled instances gathered so far, and this model is used to make predictions on all the unlabeled instances.
2. The instances for which the model is most uncertain (i.e., where its estimated probability is lowest) are given to the expert for labeling.
3. You iterate this process until the performance improvement stops being worth the labeling effort.

 
Other active learning strategies include 
- labeling the instances that would result in the largest model change 
  or 
- the largest drop in the model’s validation error
  or 
- the instances that different models disagree on (e.g., an SVM and a random forest).

## DBSCAN

This algorithm defines clusters as continuous regions of high density. 

* For each instance, the algorithm counts how many instances are located within
a small distance ε (epsilon) from it. This region is called the instance’s ε-
neighborhood.
* If an instance has at least min_samples instances in its ε-neighborhood (including
itself), then it is considered a core instance. In other words, core instances are
those that are located in dense regions.
* All instances in the neighborhood of a core instance belong to the same cluster.
This neighborhood may include other core instances; therefore, a long sequence
of neighboring core instances forms a single cluster.
* Any instance that is not a core instance and does not have one in its neighborhood
is considered an anomaly.   

This algorithm works well if all the clusters are well separated by low-density regions.
Let's try it on the moons dataset

In [62]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.05, random_state=42)
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

Just take a look at the left image.   
- DBSCAN identified quite a lot of anomalies (red crosses), plus seven different clusters (visualized by different colors). How disappointing!
- Fortunately, if we widen each instance’s neighborhood by increasing eps to 0.2, we get the clustering on the right, which looks perfect.   
   
- DBSCAN is a very simple yet powerful algorithm capable of identifying any number of clusters of any shape.   
- It is robust to outliers, and it has just two hyperparameters (eps and min_samples).  
- If the density varies significantly across the clusters, or if there’s no sufficiently low-density region around some clusters, DBSCAN can struggle to capture all the clusters properly. Moreover, its computational complexity is roughly $O(m^2n)$, so it does not scale well to large datasets.   

<img width=60% src="img/dbscan_plot.png">  

There are a few other clustering algorithms that you should have at least heard before and that are also available in Scikit-Learn.  

* Agglomerative clustering connects the nearest pair of clusters (starting from indiviual instances)   
* BIRCH (Balanced Iterative Reducing and Clustering using Hierarchies) for very large datasets   
* Mean-Shift shifts circles in the direction of higher density but it is not suited for large datasets
* Affinity propagation is similar to k-means but you don’t have to pick a number of clusters ahead of time but also not suited for large datasets   
* Spectral clustering takes a similarity matrix, reduces the matrix’s dimensionality and uses another clustering algorithm in this low-dimensional space 

# Gaussian Mixtures

- Another clustering algorithm is Gaussian mixture models, which can be used for density estimation, clustering, and anomaly detection.
- A Gaussian mixture model (GMM) is a probabilistic model that assumes that the instances were generated from a mixture of several Gaussian distributions whose parameters are unknown. 
- All the instances generated from a single Gaussian distribution form a cluster that typically looks like an ellipsoid.   
- Each cluster can have a different ellipsoidal shape, size, density, and orientation. 
- There are several GMM variants. 
- In the simplest variant, implemented in the GaussianMixture class, you must know in advance the number $k$ of Gaussian distributions.

Let's generate a dataset with three ellipsoids (K-Means had trouble with this dataset):

In [63]:
X1, y1 = make_blobs(n_samples=1000, centers=((4, -4), (0, 0)), random_state=42)
X1 = X1.dot(np.array([[0.374, 0.95], [0.732, 0.598]]))
X2, y2 = make_blobs(n_samples=250, centers=1, random_state=42)
X2 = X2 + [6, -8]
X = np.r_[X1, X2]
y = np.r_[y1, y2]

Let's train a Gaussian mixture model on the previous dataset:

In [64]:
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=3, n_init=10, random_state=42)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10, random_state=42)

Let's look at the parameters that the algorithm estimated:

In [65]:
gm.weights_

array([0.39025715, 0.40007391, 0.20966893])

- Two of the three clusters were generated with 500 instances each, while the third cluster only contains 250 instances. 
- So the true cluster weights are 0.4, 0.4, and 0.2, respectively, and that’s roughly what the algorithm found.

Did the algorithm actually converge?

In [68]:
gm.converged_

True

You can now use the model to predict which cluster each instance belongs to (hard clustering) or the probabilities that it came from each cluster. For this, just use `predict()` method or the `predict_proba()` method:

In [70]:
gm.predict(X)

array([0, 0, 1, ..., 2, 2, 2], dtype=int64)

In [71]:
gm.predict_proba(X).round(3)

array([[0.977, 0.   , 0.023],
       [0.983, 0.001, 0.016],
       [0.   , 1.   , 0.   ],
       ...,
       [0.   , 0.   , 1.   ],
       [0.   , 0.   , 1.   ],
       [0.   , 0.   , 1.   ]])

This is a generative model, so you can sample new instances from it (and get their labels):

In [72]:
X_new, y_new = gm.sample(6)
X_new

array([[-0.86944074, -0.32767626],
       [ 0.29836051,  0.28297011],
       [-2.8014927 , -0.09047309],
       [ 3.98203732,  1.49951491],
       [ 3.81677148,  0.53095244],
       [ 2.84104923, -0.73858639]])

In [73]:
y_new

array([0, 0, 1, 2, 2, 2])

Notice that they are sampled sequentially from each cluster.

Now let's plot the resulting decision boundaries (dashed lines) and density contours:

<img width=60% src="img/gaussian_mixtures_plot.png">  

## Anomaly Detection Using Gaussian Mixtures

- Gaussian Mixtures can be used for _anomaly detection_: instances located in low-density regions can be considered anomalies. 
- You must define what density threshold you want to use. 
- For example, in a manufacturing company that tries to detect defective products, the ratio of defective products is usually well-known. 
- Say it is equal to 2%, then you can set the density threshold to be the value that results in having 2% of the instances located in areas below that threshold density:

In [76]:
densities = gm.score_samples(X)
print(densities)
density_threshold = np.percentile(densities, 2)
print(density_threshold)
anomalies = X[densities < density_threshold]

[-2.60768954 -3.57110232 -3.32987086 ... -3.51347241 -4.39798588
 -3.80746532]
-5.936162762355327



<img width=60% src="img/mixture_anomaly_detection_plot.png">  